In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# na_values covers common placeholders; adjust if your competition uses special tokens
read_csv_opts = dict(low_memory=False, na_values=["", "NA", "NaN", "null"])

train = pd.read_csv('train.csv', **read_csv_opts)
test  = pd.read_csv('test.csv', **read_csv_opts)

print("train shape:", train.shape)
print("test shape:", test.shape)


train shape: (517754, 14)
test shape: (172585, 13)
